In [1]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [24]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [25]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [27]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [17]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_val_score

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [29]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-06-16 18:12:16,291] A new study created in memory with name: no-name-9a15210e-55e6-411a-b184-c73f059bc231
[I 2025-06-16 18:12:16,853] Trial 0 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 109, 'max_depth': 17}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-06-16 18:12:17,402] Trial 1 finished with value: 0.7783985102420856 and parameters: {'n_estimators': 121, 'max_depth': 17}. Best is trial 1 with value: 0.7783985102420856.
[I 2025-06-16 18:12:18,293] Trial 2 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 186, 'max_depth': 4}. Best is trial 1 with value: 0.7783985102420856.
[I 2025-06-16 18:12:18,953] Trial 3 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 139, 'max_depth': 16}. Best is trial 1 with value: 0.7783985102420856.
[I 2025-06-16 18:12:19,923] Trial 4 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 147, 'max_depth': 16}. Best is trial 1 with value: 0.77839

In [35]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'n_estimators': 73, 'max_depth': 18}
Best trial accuracy: 0.7839851024208566


In [43]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.76


In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [47]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [49]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-06-16 19:00:35,554] A new study created in memory with name: no-name-083b4662-17f5-4682-882c-43c38f06a557
[I 2025-06-16 19:00:36,006] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-06-16 19:00:36,765] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-06-16 19:00:36,986] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-06-16 19:00:37,480] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-06-16 19:00:38,033] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [51]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [53]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [55]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [57]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-06-16 19:16:23,936] A new study created in memory with name: no-name-76fec9d1-a786-4887-ad9f-69ecc344b49d
[I 2025-06-16 19:16:25,768] Trial 0 finished with value: 0.7690875232774674 and parameters: {'classifier': 'RandomForest', 'n_estimators': 219, 'max_depth': 6, 'min_samples_split': 5, 'min_samples_leaf': 4, 'bootstrap': False}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-06-16 19:16:28,430] Trial 1 finished with value: 0.7635009310986964 and parameters: {'classifier': 'RandomForest', 'n_estimators': 256, 'max_depth': 7, 'min_samples_split': 7, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-06-16 19:16:29,275] Trial 2 finished with value: 0.7597765363128491 and parameters: {'classifier': 'RandomForest', 'n_estimators': 98, 'max_depth': 15, 'min_samples_split': 6, 'min_samples_leaf': 4, 'bootstrap': False}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-06-16 19:16:29,323] Trial 3 finished with value: 0.

In [59]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.1182041260628474, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [61]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.769088,2025-06-16 19:16:23.939700,2025-06-16 19:16:25.768406,0 days 00:00:01.828706,NaN,False,RandomForest,NaN,NaN,NaN,6.0,4.0,5.0,219.0,COMPLETE
1,1,0.763501,2025-06-16 19:16:25.770417,2025-06-16 19:16:28.430473,0 days 00:00:02.660056,NaN,True,RandomForest,NaN,NaN,NaN,7.0,4.0,7.0,256.0,COMPLETE
2,2,0.759777,2025-06-16 19:16:28.431477,2025-06-16 19:16:29.275751,0 days 00:00:00.844274,NaN,False,RandomForest,NaN,NaN,NaN,15.0,4.0,6.0,98.0,COMPLETE
3,3,0.733706,2025-06-16 19:16:29.276775,2025-06-16 19:16:29.323901,0 days 00:00:00.047126,1.794065,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.743017,2025-06-16 19:16:29.325383,2025-06-16 19:16:35.457426,0 days 00:00:06.132043,NaN,NaN,GradientBoosting,NaN,NaN,0.203531,19.0,2.0,2.0,146.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.785847,2025-06-16 19:17:35.035403,2025-06-16 19:17:35.100604,0 days 00:00:00.065201,0.312153,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.761639,2025-06-16 19:17:35.101512,2025-06-16 19:17:37.429008,0 days 00:00:02.327496,NaN,True,RandomForest,NaN,NaN,NaN,20.0,9.0,9.0,230.0,COMPLETE
97,97,0.789572,2025-06-16 19:17:37.431237,2025-06-16 19:17:37.495606,0 days 00:00:00.064369,0.120463,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.744879,2025-06-16 19:17:37.497168,2025-06-16 19:17:37.575226,0 days 00:00:00.078058,0.558606,NaN,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
